# Polars TI ([polars-ti](https://github.com/CMobley7/polars-ti)) Studies for Custom Technical Analysis

## Topics
- What is a Polars TI Study?
    - Builtin Studies: __AllStudy__ and __CommonStudy__
    - Creating Studies
- Watchlist Class
    - Study Management and Execution
    - **NOTE:** The **watchlist** module is independent of Polars TI. To easily use it, copy it from your local polars-ti installation directory into your project directory.
- Indicator Composition/Chaining for more Complex Studies
    - Comprehensive Example: _MACD and RSI Momo with BBANDS and SMAs 50 & 200 and Cumulative Log Returns_

In [ ]:
%matplotlib inline
from datetime import datetime
import pandas as pd
import polars_ti as ti
from alphaVantageAPI.alphavantage import AlphaVantage  # pip install alphaVantage-api
from watchlist import Watchlist  # Is this failing? If so, copy it locally. See above.

print(
    f"\nPolars TI v{ti.version}\nTo install the Latest Version:\n$ pip install -U git+https://github.com/CMobley7/polars-ti\n"
)
%pylab inline

# What is a Polars TI Study?
A _Study_ is a simple way to name and group technical indicators. Technically, a _Study_ is a simple Data Class to contain list of indicators and their parameters. __Note__: _Study_ is experimental and subject to change. Polars TI comes with two basic Studies: __AllStudy__ and __CommonStudy__.

## Study Requirements:
- _name_: Some short memorable string.  _Note_: Case-insensitive "All" is reserved.
- _ti_: A list of dicts containing keyword arguments to identify the indicator and the indicator's arguments

## Optional Requirements:
- _description_: A more detailed description of what the Study tries to capture. Default: None
- _created_: At datetime string of when it was created. Default: Automatically generated.

### Things to note:
- A Study will __fail__ when consumed by Polars TI if there is no {"kind": "indicator name"} attribute.

# Builtin Examples

### All
Default Values

In [ ]:
AllStudy = ti.AllStudy
print(f"{AllStudy.name = }")
print(f"{AllStudy.description = }")
print(f"{AllStudy.created = }")
print(f"{AllStudy.ti = }")
print(f"{AllStudy.cores = }")

### Common
Default Values

In [ ]:
CommonStudy = ti.CommonStudy
print(f"{CommonStudy.name = }")
print(f"{CommonStudy.description = }")
print(f"{CommonStudy.created = }")
print(f"{CommonStudy.ti = }")
print(f"{CommonStudy.cores = }")

# Creating Studies
Studies require a **name** and an array of dicts containing the "kind" of indicator ("sma") and other potential parameters for the analysis.

### Simple Study A

In [ ]:
custom_a = ti.Study(
    name="A",
    cores=0,
    ti=[{"kind": "sma", "length": 50}, {"kind": "sma", "length": 200}],
)
custom_a

### Simple Study B

In [ ]:
custom_b = ti.Study(
    name="B",
    cores=0,
    ti=[
        {"kind": "ema", "length": 8},
        {"kind": "ema", "length": 21},
        {"kind": "log_return", "cumulative": True},
        {"kind": "rsi"},
        {"kind": "supertrend"},
    ],
)
custom_b

### Bad Study. (Misspelled Indicator)

In [ ]:
# Misspelled indicator, will fail later when ran with Polars TI
custom_run_failure = ti.Study(
    name="Runtime Failure", cores=0, ti=[{"kind": "peret_return"}]
)
custom_run_failure

# Study Management and Execution with _Watchlist_

### Initialize AlphaVantage Data Source

In [ ]:
AV = AlphaVantage(
    api_key="YOUR API KEY",
    premium=False,
    output_size="full",
    clean=True,
    export_path=".",
    export=True,
)
AV

### Create Watchlist and set it's 'ds' to AlphaVantage

In [8]:
data_source = "av"  # Default
data_source = "yahoo"
watch = Watchlist(["SPY", "IWM"], ds_name=data_source, timed=True)



#### Info about the Watchlist. Note, the default Study is "All"

In [ ]:
watch

### Help about Watchlist

In [ ]:
help(Watchlist)

### Default Study is "Common"

In [ ]:
# No arguments loads all the tickers and applies the Study to each ticker.
# The result can be accessed with Watchlist's 'data' property which returns a
# dictionary keyed by ticker and DataFrames as values
watch.load(verbose=True)

In [ ]:
", ".join([f"{t}: {d.shape}" for t, d in watch.data.items()])

In [ ]:
watch.data["SPY"]

In [ ]:
watch.load("SPY", plot=True, mas=True)

## Easy to swap Studies and run them

### Running Simple Study A

In [ ]:
# Load custom_a into Watchlist and verify
watch.study = custom_a
watch.study

In [ ]:
watch.load("IWM")

### Running Simple Study B

In [ ]:
# Load custom_b into Watchlist and verify
watch.study = custom_b
watch.study

In [ ]:
watch.load("IWM")

### Running Bad Study. (Misspelled indicator)

In [ ]:
# Load custom_run_failure into Watchlist and verify
watch.study = custom_run_failure
watch.study

In [ ]:
try:
    iwm = watch.load("IWM")
except AttributeError as error:
    print(f"[X] Oops! {error}")

# Indicator Composition/Chaining
- When you need an indicator to depend on the value of a prior indicator
- Utilitze _prefix_ or _suffix_ to help identify unique columns or avoid column name clashes.
- Set ```cores=0``` for better performance when few indicators

### Volume MAs and MA chains

In [ ]:
# Set EMA's and SMA's 'close' to 'volume' to create Volume MAs, prefix 'volume' MAs with 'VOLUME' so easy to identify the column
# Take a price EMA and apply LINREG from EMA's output
volmas_price_ma_chain = [
    {"kind": "ema", "close": "volume", "length": 10, "prefix": "VOLUME"},
    {"kind": "sma", "close": "volume", "length": 20, "prefix": "VOLUME"},
    {"kind": "ema", "length": 5},
    {"kind": "linreg", "close": "EMA_5", "length": 8, "prefix": "EMA_5"},
]
vp_ma_chain_ti = ti.Study(
    "Volume MAs and Price MA chain", cores=0, ti=volmas_price_ma_chain
)
vp_ma_chain_ti

In [ ]:
# Update the Watchlist
watch.study = vp_ma_chain_ti
watch.study.name

In [ ]:
spy = watch.load("SPY")
spy

### MACD BBANDS

In [ ]:
# MACD is the initial indicator that BBANDS depends on.
# Set BBANDS's 'close' to MACD's main signal, in this case 'MACD_12_26_9' and add a prefix (or suffix) so it's easier to identify
macd_bands_ti = [
    {"kind": "macd"},
    {
        "kind": "bbands",
        "close": "MACD_12_26_9",
        "length": 20,
        "ddof": 0,
        "prefix": "MACD",
    },
]
macd_bands_ti = ti.Study(
    "MACD BBands",
    cores=0,
    ti=macd_bands_ti,
    description=f"BBANDS_{macd_bands_ti[1]['length']} applied to MACD",
)
macd_bands_ti

In [ ]:
# Update the Watchlist
watch.study = macd_bands_ti
watch.study.name

In [ ]:
spy = watch.load("SPY")
spy

# Comprehensive Study

### MACD and RSI Momentum with BBANDS and SMAs and Cumulative Log Returns

In [ ]:
momo_bands_sma_ti = [
    {"kind": "sma", "length": 50},
    {"kind": "sma", "length": 200},
    {"kind": "bbands", "length": 20, "ddof": 0},
    {"kind": "macd"},
    {"kind": "rsi"},
    {"kind": "log_return", "cumulative": True},
    {"kind": "sma", "close": "CUMLOGRET_1", "length": 5, "suffix": "CUMLOGRET"},
]
momo_bands_sma_Study = ti.Study(
    name="Momo, Bands and SMAs and Cumulative Log Returns",  # name
    ti=momo_bands_sma_ti,  # ti
    description="MACD and RSI Momo with BBANDS and SMAs 50 & 200 and Cumulative Log Returns",  # description
    cores=0,
)
momo_bands_sma_Study

In [ ]:
# Update the Watchlist
watch.study = momo_bands_sma_Study
watch.study.name

In [ ]:
spy = watch.load("SPY")
# Apply constants to the DataFrame for indicators
spy.ti.constants(True, [0, 30, 70])
spy.tail()

# Additional Study Options

The ```params``` keyword takes a _tuple_ as a shorthand to the parameter arguments in order.
* **Note**: If the indicator arguments change, so will results. Breaking Changes will **always** be posted on the README.

The ```col_numbers``` keyword takes a _tuple_ specifying which column to return if the result is a DataFrame.

In [ ]:
params_ti = [
    {"kind": "ema", "params": (10,)},
    # params sets MACD's keyword arguments: fast=9, slow=19, signal=10
    # and returning the 2nd column: histogram
    {"kind": "macd", "params": (9, 19, 10), "col_numbers": (1,)},
    # Selects the Lower and Upper Bands and renames them LB and UB, ignoring the MB
    {"kind": "bbands", "col_numbers": (0, 2), "col_names": ("LB", "UB")},
    {"kind": "log_return", "params": (5, False)},
]
params_ti_Study = ti.Study(
    name="EMA, MACD History, Outter BBands, Log Returns",  # name
    ti=params_ti,  # ti
    description="EMA, MACD History, BBands(LB, UB), and Log Returns Study",  # description
    cores=0,
)
params_ti_Study

In [ ]:
# Update the Watchlist
watch.study = params_ti_Study
watch.study.name

In [ ]:
spy = watch.load("SPY")
spy.tail()

# Disclaimer
* All investments involve risk, and the past performance of a security, industry, sector, market, financial product, trading Study, or individual’s trading does not guarantee future results or returns. Investors are fully responsible for any investment decisions they make. Such decisions should be based solely on an evaluation of their financial circumstances, investment objectives, risk tolerance, and liquidity needs.

* Any opinions, news, research, analyses, prices, or other information offered is provided as general market commentary, and does not constitute investment advice. I will not accept liability for any loss or damage, including without limitation any loss of profit, which may arise directly or indirectly from use of or reliance on such information.